In [1]:
from sqlalchemy import create_engine
import yaml
import logging
log = logging.getLogger(__name__)
from sqlalchemy import Column, ForeignKey, Boolean, String, \
                       Integer, BigInteger, Float, Date    
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import Enum, UniqueConstraint
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
#import models
import enum

In [2]:
config_file_path = '../../../data/psql_creds.yaml'

with open(config_file_path, 'r') as f:
        vals = yaml.safe_load(f)

url = f"postgresql://{vals['PGUSER']}:{vals['PGPASSWORD']}@{vals['PGHOST']}:{vals['PGPORT']}/{vals['PGDATABASE']}"

try:
    engine = create_engine(url, pool_size = 50, echo=True)
    log.info(f"Connected to {vals['PGDATABASE']} database!")
except IOError:
    log.exception("Failed to get database connection!")


In [17]:
Base = declarative_base()

# key value store (essentially you're going to constrain the values in the frequency column
# of the price table to be one of these values)
# see: https://docs.sqlalchemy.org/en/14/core/type_basics.html
class PriceFrequency(enum.Enum):
    daily = 'daily'
    weekly = 'weekly'
    monthly = 'monthly'
    quarterly = 'quarterly'
    yearly = 'yearly'

# the security information downloaded from the nasdaq website
class security_info(Base):
    __tablename__ = 'security_info'
    id = Column(Integer, primary_key=True, autoincrement=True)
    ticker = Column('ticker', String(12), nullable=False)
    name = Column('name', String(200), nullable=False)
    exchange = Column('exchange', String(200), nullable=False)
    test_security = Column('test_security', Boolean)
    round_lot = Column('round_lot', Float)

# equity price data which for now comes from yfinance
# This has a many-to-one relationship to security info (since many instances of security obsercations are 
# associated with one instance of security info)
class security_price_observations(Base):
    __tablename__ = 'security_price_observations'
    id = Column(Integer, primary_key=True)
    date = Column('date', Date, nullable=False)
    open = Column('open', Float)
    high = Column('high', Float)
    low = Column('low', Float)
    close = Column('close', Float)
    volume = Column('volume', BigInteger)
    adj_close = Column('adj_close', Float)
    frequency = Column('frequency', Enum(PriceFrequency),nullable=False)
    # setting foreign key and its constraints. onupdate="CASCADE" simply means
    # if we update the security info table, this update would cascade to this table.
    # ondelete="CASCADE" is dangerous because if we delete a stock in the security info
    # table, its whole history will be deleted here. Maybe best to comment out.. 
    # see: https://stackoverflow.com/questions/6720050/foreign-key-constraints-when-to-use-on-update-and-on-delete
    security_id = Column(Integer, ForeignKey('security_info.id',
                                    onupdate="CASCADE"),
                                    #ondelete="CASCADE"),
                                    nullable=False)
    # ensure no duplicate values of date and security id will be inserted
    UniqueConstraint('date', 'security_id')
    # in the documentation, it says to place the relationship definition
    # in the parent. If we consider this as a many-to-one relationship,
    # then it makes sense. 
    security_info = relationship('security_info')

In [16]:
# create an ORM session factory bound to the engine
Session = sessionmaker(bind=engine)
meta = MetaData(bind=engine)
# extract new session from the session factory
session = Session()
# generate the schema
Base.metadata.create_all(engine)

2021-09-25 16:34:43,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-25 16:34:43,634 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-09-25 16:34:43,635 INFO sqlalchemy.engine.Engine [cached since 176.2s ago] {'name': 'security'}
2021-09-25 16:34:43,638 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-09-25 16:34:43,639 INFO sqlalchemy.engine.Engine [cached since 176.2s ago] {'name': 'security_price'}
2021-09-25 16:34:43,641 INFO sqlalchemy.engine.Engine 
            SELECT EXISTS (
                SELECT * FROM pg_catalog.pg_type t
                WHERE t.typname = %(typname)s
                AND pg_type_is_visible(t.oid)
                )
                
2021-09-25 16:34:43,642 INFO sqlalchemy.engine.Engine [cached since 176.